E2 Intermediate

Brian Bannon

In [35]:
import numpy as np
import pandas as pd

In [36]:
def calculate_single_moment(bridge_length: float, load: float, load_position: float, x: float) -> float:
    """
    Function to calculate the internal bending moment at a specific point along a simply supported bridge due to a single point load.
    :param bridge_length: Total length of the bridge in meters.
    :param load: Magnitude of the applied point load in kilonewtons (kN).
    :param load_position: Distance from the left support to the applied load in meters.
    :param x: Position along the bridge where the bending moment is calculated in meters.
    :return: Bending moment at position x in kilonewton-meters (kN-m).
    """
    if x <= load_position:
        b = bridge_length - load_position
        return (load * b * x) / bridge_length
    else:
        return (load * load_position * (bridge_length - x)) / bridge_length


In [37]:
def calculate_multiple_moments(bridge_length: float, loads: list[float], positions: list[float], x: float) -> float:
    """
    Calculates the total bending moment at a specific position x on a simply supported bridge due to multiple point loads using superposition
    :param bridge_length: Length of the bridge span in meters
    :param loads: List of point load magnitudes in kilonewtons
    :param positions: List of distances from the left support where each load is applied in meters
    :param x: Position along the bridge where the bending moment is calculated, measured from the left support in meters
    :return: Total bending moment at position x in kilonewton-meters
    """
    total_moment = 0

    for i, j in zip(loads, positions):
        moment = calculate_single_moment(bridge_length, i, j, x)
        total_moment += moment

    return total_moment

In [38]:
def find_max_moment_multiple(bridge_length: float, loads: list[float], positions: list[float]) -> tuple[float, float]:
    """
    Finds the maximum bending moment on a simply supported bridge under multiple point loads
    and the position where it occurs.
    :param bridge_length: Span of the bridge in meters
    :param loads: List of point load magnitudes in kN
    :param positions: List of distances from the left support where each load is applied
    :return: Tuple containing (max_moment in kNm, position_at_max_moment in meters)
    """

    test_points = list(np.linspace(0, bridge_length, 101))

    valid_positions = [p for p in positions if 0 <= p <= bridge_length]

    all_points = sorted(set(test_points + valid_positions))

    max_moment = float('-inf')
    position_at_max_moment = 0

    for x in all_points:
        moment = calculate_multiple_moments(bridge_length, loads, positions, x)
        if moment > max_moment:
            max_moment = moment
            position_at_max_moment = x

    return max_moment, position_at_max_moment

In [39]:
def create_vehicle_loads(vehicle_type: str, front_position: float) -> tuple[list[float], list[float]]:
    """
    Generates realistic axle loads and positions for a vehicle on a bridge.
    :param vehicle_type: One of 'car', 'bus', or 'truck'
    :param front_position: Location of the front axle from the left support in meters
    :return: Tuple containing loads list and positions list
    """

    if vehicle_type == 'car':
        loads = [6, 9]
        spacings = [2.5]
    elif vehicle_type == 'bus':
        loads = [45, 75]
        spacings = [6.0]
    else:
        loads = [50, 70, 70]
        spacings = [3.5, 1.4]

    positions = [front_position]
    for s in spacings:
        positions.append(positions[-1] - s)

    return loads, positions

In [40]:
def analyze_vehicle_crossing(bridge_length: float, vehicle_type: str, num_positions: int = 50) -> dict[str, object]:
    """
    Simulates a vehicle crossing a bridge to find the critical front axle position
    that produces the maximum bending moment.
    :param bridge_length: Span of the bridge in meters
    :param vehicle_type: Type of vehicle
    :param num_positions: Number of front axle positions to test
    :return: Dictionary with keys:
             - 'critical_front_position': position of front axle causing max moment
             - 'max_moment': highest bending moment found
             - 'max_moment_location': x-location on bridge of max moment
             - 'all_maxima': list of maximum moments for each vehicle position
    """

    start_pos = -10.0
    end_pos = bridge_length + 10.0

    front_positions = list(np.linspace(start_pos, end_pos, num_positions))

    all_maxima: list[float] = []
    max_moment = float('-inf')
    critical_front_position = 0.0
    max_moment_location = 0.0

    for front_pos in front_positions:
        loads, positions = create_vehicle_loads(vehicle_type, front_pos)

        moment, location = find_max_moment_multiple(bridge_length, loads, positions)

        all_maxima.append(moment)

        if moment > max_moment:
            max_moment = moment
            critical_front_position = front_pos
            max_moment_location = location

    return {
        'critical_front_position': critical_front_position,
        'max_moment': max_moment,
        'max_moment_location': max_moment_location,
        'all_maxima': all_maxima
    }

In [41]:
def create_convoy_load(length, vehicles):
   for v in vehicles:
       vehicle_dict = analyze_vehicle_crossing(length, v)

In [42]:
def display_data(d: dict[str, tuple]) -> pd.DataFrame:
    df = pd.DataFrame(d)
    length = d['Value'][1]
    vehicle = d['Value'][3]
    vehicle_dict = analyze_vehicle_crossing(length, vehicle)
    front = vehicle_dict['critical_front_position']
    max_moment_actual = vehicle_dict['max_moment']
    df.loc[len(df)] = ('Maximum bending moment', max_moment_actual)
    df.loc[len(df)] = ('Position of maximum moment', vehicle_dict['max_moment_location'])
    df.loc[len(df)] = ('Critical front axle position', front)
    loads, positions = create_vehicle_loads(vehicle, front)
    total = 0
    d_t = 0
    for d, p in zip(loads, [0] + positions):
        d_t += d
        total += p * d_t
    centroid = total/sum(positions)
    single_load_moment = calculate_single_moment(length, sum(loads), front-centroid, length/2)
    df.loc[len(df)] = ('Centroid', centroid)
    df.loc[len(df)] = ('Moment from centroid', single_load_moment)
    df.loc[len(df)] = ('Load amplification factor', max_moment_actual / single_load_moment)
    return df

In [43]:
data: dict[str, tuple] = {
    'Parameter': ('Name', 'Bridge length (m)', 'Bridge Type', 'Vehicle type (s)'),
    'Value': ('City Bus on Urban Bridge', 25, 'Reinforced Concrete', ['bus'])
}
display_data(data)

,Parameter,Value
0,Name,City Bus on Urban Bridge
1,Bridge length (m),25
2,Bridge Type,Reinforced Concrete
3,Vehicle type (s),[bus]
4,Maximum bending moment,1051.259592
5,Position of maximum moment,12.214286
6,Critical front axle position,15.714286
7,Centroid,108.573009
8,Moment from centroid,-8821.578698
9,Load amplification factor,-0.119169
